In [1]:
import pandas as pd
import numpy as np
import warnings
from functools import reduce
from dateutil import relativedelta

warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [2]:
def intck_month( start, end ):
    rd = relativedelta.relativedelta( pd.to_datetime( end ), pd.to_datetime( start ) )
    return rd.months

In [3]:
df = pd.read_csv("D:/Varthana/MOM_DATA_28-Sep-20.csv")

In [4]:
df.columns = df.columns.str.lower()

### Business date conversion to date format

In [5]:
df['dt_businessdate'] = pd.to_datetime(df['dt_businessdate'])
df.shape

(153571, 29)

### Flagging Education loans and excluding

In [6]:
df["EL_Flag"] = np.where(df['sz_loan_account_no'].str[-2:] == "EL", 1,0)

In [7]:
df["EL_Flag"].value_counts()

0    153571
Name: EL_Flag, dtype: int64

In [8]:
df[df["EL_Flag"]==0].shape

(153571, 30)

##### Keeping only the records with School loans by excluding the Education loans

In [9]:
df_1 = df[df["EL_Flag"]==0]

#### excluding the old accounts


In [10]:
df_x = df_1[df_1["sz_loan_account_no"].str.len()>9]

#### Data subset with only necessary columns

In [11]:
df_2=pd.DataFrame(df_x[['sz_loan_account_no','dt_businessdate','i_dpd',"sz_delinquency_str" ]])

In [12]:
df_2 = df_2.sort_values(by = ['sz_loan_account_no','dt_businessdate'])
df_2 = df_2.reset_index()

### Creating 30+, 60+ & 90+ flags

In [13]:
df_2["30Plus_flag"] = np.where((df_2["i_dpd"]>=30),1,0)
df_2["60Plus_flag"] = np.where((df_2["i_dpd"]>=60),1,0)
df_2["90Plus_flag"] = np.where((df_2["i_dpd"]>=90),1,0)
df_2["XPlus_flag"] = np.where((df_2["i_dpd"]>0),1,0)

In [14]:
df_2["30Plus_flag"].value_counts()

0    72989
1     4073
Name: 30Plus_flag, dtype: int64

In [15]:
df_2["60Plus_flag"].value_counts()

0    74316
1     2746
Name: 60Plus_flag, dtype: int64

In [16]:
df_2["90Plus_flag"].value_counts()

0    75760
1     1302
Name: 90Plus_flag, dtype: int64

In [17]:
#df_2["MOB"] = df_2.groupby('sz_loan_account_no')['dt_businessdate'].rank(method='first')

In [18]:
df_2["MaxBusDate"] = max(df_2.dt_businessdate)

### LOS Data import to include disb date

In [19]:
LOS_Base =  pd.read_excel("D:/Varthana/PIPELINE (1.xlsx")

In [20]:
LOS_Base["DISBURSEMENT_DATE"] = pd.to_datetime(LOS_Base["DISBURSEMENT_DATE"]) 
LOS_Base.shape

(11311, 47)

In [21]:
LOS_temp = pd.DataFrame(LOS_Base[["SZ_APPLICATION_NO","DISBURSEMENT_DATE"]])

In [22]:
LOS_temp = LOS_temp.sort_values("DISBURSEMENT_DATE",ascending=True )

In [23]:
LOS_temp_1 = LOS_temp.drop_duplicates('SZ_APPLICATION_NO').sort_index()

In [24]:
df_3  = pd.merge(df_2, LOS_temp_1,how="left",left_on="sz_loan_account_no", right_on="SZ_APPLICATION_NO")

In [25]:
df_3["nb_mths"] =  round(((df_3["MaxBusDate"] - df_3["DISBURSEMENT_DATE"])/np.timedelta64(1, 'M')))

In [26]:
df_3["MOB"] =  round(((df_3["dt_businessdate"] - df_3["DISBURSEMENT_DATE"])/np.timedelta64(1, 'M')))

In [27]:
# df_3["nb_mths"] = round(df_3['nb_mths'])
# df_3["MOB"] = round(df_3['MOB'])

In [28]:
df_30plusever_3M = df_3[(df_3["MOB"]<=3) & (df_3["nb_mths"]>3)].groupby('sz_loan_account_no').agg(ever_30plus_3m = ("30Plus_flag","max"))

In [29]:
df_3['disb_year'] = pd.DatetimeIndex(df_3['DISBURSEMENT_DATE']).year

## Bad rate creation for business case - 90+ in 12 MOB for final model 

In [30]:
df_xx = pd.merge(df_3,df_x.loc[:,["sz_loan_account_no","dt_businessdate","f_future_principal"]]
         ,how = "left", on = ["sz_loan_account_no","dt_businessdate"])

##### LMS data overview to get the loan amoutn

In [31]:
LMS_det_all = pd.read_csv('D:/Varthana/LMS_DETAILS_11-Sep-20.csv')

In [32]:
df_xxx = pd.merge(df_xx, LMS_det_all.loc[:,["LOAN_ID","LOAN_AMOUNT"]],how="left", left_on = "sz_loan_account_no", right_on = "LOAN_ID")

In [33]:
df_4 = df_xxx[(df_xxx['dt_businessdate'] <= '2020-02-29') ]
df_4.shape

(53729, 18)

## Data for roll rate analysis


In [34]:
df_2018_orig = df_xxx[df_xxx['disb_year'] == 2018]

In [35]:
#df_2018_orig.to_csv('D:/Varthana/MoM_2018_origdata.csv')

### perf calculation for impact analysis

In [36]:
df_temp = df_4.sort_values(by = ['sz_loan_account_no','dt_businessdate'], ascending=[True, False])

In [37]:
df_temp_2 = df_temp.drop_duplicates('SZ_APPLICATION_NO').reset_index()

In [38]:
df_3_temp = df_3[df_3['dt_businessdate'] <= '2020-02-29']

In [39]:
aggreg_12_90_2018 = df_3_temp[ (df_3_temp["disb_year"] == 2018) ].groupby('sz_loan_account_no').agg(ever_90p_12m = ("90Plus_flag","max"))
aggreg_12_90_2018.reset_index(inplace=True)

In [40]:
aggreg_12_60_2018 = df_3_temp[(df_3_temp["disb_year"] == 2018) ].groupby('sz_loan_account_no').agg(ever_60p_12m = ("60Plus_flag","max"))
aggreg_12_60_2018.reset_index(inplace=True)

In [41]:
aggreg_12_90_2018_v2 = pd.merge(aggreg_12_90_2018,df_temp_2.loc[:,["sz_loan_account_no",
                                                                 "MOB",
                                                                 "f_future_principal",
                                                                 "LOAN_AMOUNT"]],
                                how = "left",
                                on="sz_loan_account_no")

In [42]:
aggreg_12_60_2018_v2 = pd.merge(aggreg_12_60_2018,df_temp_2.loc[:,["sz_loan_account_no",
                                                                 "MOB",
                                                                 "f_future_principal",
                                                                 "LOAN_AMOUNT"]],
                                how = "left",
                                on="sz_loan_account_no")

In [43]:
def perf(MOB_perf, n_mths, flag):
    df1 = df_xxx[(df_xxx["MOB"] <=MOB_perf) & (df_xxx["nb_mths"] >n_mths)]
    df_temp = df1.sort_values(by = ['sz_loan_account_no','dt_businessdate'], ascending=[True, False])
    df_temp_2 = df_temp.drop_duplicates('SZ_APPLICATION_NO').reset_index()  
    aggreg_90 = df_3[(df_3["MOB"] <=MOB_perf) & (df_3["nb_mths"] >n_mths) ].groupby('sz_loan_account_no').agg(ever_90p = (flag,"max"))
    aggreg_90.reset_index(inplace=True)
    aggreg_90_v2 = pd.merge(aggreg_90,df_temp_2.loc[:,["sz_loan_account_no",
                                                                     "MOB",
                                                                     "f_future_principal",
                                                                     "LOAN_AMOUNT"]],
                                    how = "left",
                                    on="sz_loan_account_no")
    return aggreg_90_v2

In [44]:
aggreg_15_90_9m_v2 = perf(15,9,"90Plus_flag")

In [45]:
aggreg_18_90_9m_v2 = perf(18,9,"90Plus_flag")

In [46]:
aggreg_24_90_9m_v2 = perf(24,9,"90Plus_flag")

In [47]:
aggreg_12_90_9m_v2 = perf(12,9,"90Plus_flag")

In [48]:
aggreg_12_90_9m_v2["ever_90p"].value_counts()

0    3942
1      46
Name: ever_90p, dtype: int64

In [49]:
aggreg_12_90_9m_v3  = aggreg_12_90_9m_v2.rename(columns = {"ever_90p": "ever_90p_12m",
                                                          "MOB":"MOB_12m",
                                                          "f_future_principal":"f_future_principal_12m",
                                                          "LOAN_AMOUNT":"LOAN_AMOUNT_12m"})

aggreg_15_90_9m_v3  = aggreg_15_90_9m_v2.rename(columns = {"ever_90p": "ever_90p_15m",
                                                          "MOB":"MOB_15m",
                                                          "f_future_principal":"f_future_principal_15m",
                                                          "LOAN_AMOUNT":"LOAN_AMOUNT_15m"})
aggreg_18_90_9m_v3  = aggreg_18_90_9m_v2.rename(columns = {"ever_90p": "ever_90p_18m",
                                                          "MOB":"MOB_18m",
                                                          "f_future_principal":"f_future_principal_18m",
                                                          "LOAN_AMOUNT":"LOAN_AMOUNT_18m"})
aggreg_24_90_9m_v3  = aggreg_24_90_9m_v2.rename(columns = {"ever_90p": "ever_90p_24m",
                                                          "MOB":"MOB_24m",
                                                          "f_future_principal":"f_future_principal_24m",
                                                          "LOAN_AMOUNT":"LOAN_AMOUNT_24m"})

In [50]:
mer_list = [aggreg_12_90_9m_v3,aggreg_15_90_9m_v3,aggreg_18_90_9m_v3,aggreg_24_90_9m_v3]

In [51]:
df_merged_perf = reduce(lambda  left,right: pd.merge(left,right,on=['sz_loan_account_no'],
                                            how='outer'), mer_list)

In [52]:
#df_merged_perf.to_csv('/home/siddarth/LOS_data/aggreg_12151824_mob_perf_9mob_data.csv')

### Function for getting the bad rates as required


In [53]:
def badrates_ever(data, mob, dpd_flag):
    aggreg = data[(data["MOB"]<=mob) & (data["nb_mths"]>mob) ].groupby('sz_loan_account_no').agg(ever_dpd = (dpd_flag,"max"))
    aggreg.reset_index(inplace=True)
#     aggreg.value_counts()
    return aggreg
    

In [54]:
print(badrates_ever(df_3,6,"XPlus_flag")["ever_dpd"].value_counts())

0    4026
1     276
Name: ever_dpd, dtype: int64


In [55]:
aggreg_30_3 = badrates_ever(df_3,3,"30Plus_flag")
aggreg_60_3 = badrates_ever(df_3,3,"60Plus_flag")
aggreg_90_3 = badrates_ever(df_3,3,"90Plus_flag")
aggreg_X_3 = badrates_ever(df_3,3,"XPlus_flag")

aggreg_30_6 = badrates_ever(df_3,6,"30Plus_flag")
aggreg_60_6 = badrates_ever(df_3,6,"60Plus_flag")
aggreg_90_6 = badrates_ever(df_3,6,"90Plus_flag")
aggreg_X_6 = badrates_ever(df_3,6,"XPlus_flag")

aggreg_30_9 = badrates_ever(df_3,9,"30Plus_flag")
aggreg_60_9 = badrates_ever(df_3,9,"60Plus_flag")
aggreg_90_9 = badrates_ever(df_3,9,"90Plus_flag")
aggreg_X_9 = badrates_ever(df_3,9,"XPlus_flag")

aggreg_30_12 = badrates_ever(df_3,12,"30Plus_flag")
aggreg_60_12 = badrates_ever(df_3,12,"60Plus_flag")
aggreg_90_12 = badrates_ever(df_3,12,"90Plus_flag")
aggreg_X_12 = badrates_ever(df_3,12,"XPlus_flag")

aggreg_30_15 = badrates_ever(df_3,15,"30Plus_flag")
aggreg_60_15 = badrates_ever(df_3,15,"60Plus_flag")
aggreg_90_15 = badrates_ever(df_3,15,"90Plus_flag")
aggreg_X_15 = badrates_ever(df_3,15,"XPlus_flag")

aggreg_30_18 = badrates_ever(df_3,18,"30Plus_flag")
aggreg_60_18 = badrates_ever(df_3,18,"60Plus_flag")
aggreg_90_18 = badrates_ever(df_3,18,"90Plus_flag")
aggreg_X_18 = badrates_ever(df_3,18,"XPlus_flag")

aggreg_30_24 = badrates_ever(df_3,24,"30Plus_flag")
aggreg_60_24 = badrates_ever(df_3,24,"60Plus_flag")
aggreg_90_24 = badrates_ever(df_3,24,"90Plus_flag")
aggreg_X_24 = badrates_ever(df_3,24,"XPlus_flag")


In [56]:
aggreg_30_3  = aggreg_30_3.rename(columns = {"ever_dpd": "ever_dpd_30_3"})
aggreg_60_3  = aggreg_60_3.rename(columns = {"ever_dpd": "ever_dpd_60_3"})
aggreg_90_3  = aggreg_90_3.rename(columns = {"ever_dpd": "ever_dpd_90_3"})
aggreg_X_3  = aggreg_X_3.rename(columns = {"ever_dpd": "ever_dpd_X_3"})
aggreg_30_6  = aggreg_30_6.rename(columns = {"ever_dpd": "ever_dpd_30_6"})
aggreg_60_6  = aggreg_60_6.rename(columns = {"ever_dpd": "ever_dpd_60_6"})
aggreg_90_6  = aggreg_90_6.rename(columns = {"ever_dpd": "ever_dpd_90_6"})
aggreg_X_6  = aggreg_X_6.rename(columns = {"ever_dpd": "ever_dpd_X_6"})
aggreg_30_9  = aggreg_30_9.rename(columns = {"ever_dpd": "ever_dpd_30_9"})
aggreg_60_9  = aggreg_60_9.rename(columns = {"ever_dpd": "ever_dpd_60_9"})
aggreg_90_9  = aggreg_90_9.rename(columns = {"ever_dpd": "ever_dpd_90_9"})
aggreg_X_9  = aggreg_X_9.rename(columns = {"ever_dpd": "ever_dpd_X_9"})
aggreg_30_12  = aggreg_30_12.rename(columns = {"ever_dpd": "ever_dpd_30_12"})
aggreg_60_12  = aggreg_60_12.rename(columns = {"ever_dpd": "ever_dpd_60_12"})
aggreg_90_12  = aggreg_90_12.rename(columns = {"ever_dpd": "ever_dpd_90_12"})
aggreg_X_12  = aggreg_X_12.rename(columns = {"ever_dpd": "ever_dpd_X_12"})
aggreg_30_15  = aggreg_30_15.rename(columns = {"ever_dpd": "ever_dpd_30_15"})
aggreg_60_15  = aggreg_60_15.rename(columns = {"ever_dpd": "ever_dpd_60_15"})
aggreg_90_15  = aggreg_90_15.rename(columns = {"ever_dpd": "ever_dpd_90_15"})
aggreg_X_15  = aggreg_X_15.rename(columns = {"ever_dpd": "ever_dpd_X_15"})
aggreg_30_18  = aggreg_30_18.rename(columns = {"ever_dpd": "ever_dpd_30_18"})
aggreg_60_18  = aggreg_60_18.rename(columns = {"ever_dpd": "ever_dpd_60_18"})
aggreg_90_18  = aggreg_90_18.rename(columns = {"ever_dpd": "ever_dpd_90_18"})
aggreg_X_18  = aggreg_X_18.rename(columns = {"ever_dpd": "ever_dpd_X_18"})
aggreg_30_24  = aggreg_30_24.rename(columns = {"ever_dpd": "ever_dpd_30_24"})
aggreg_60_24  = aggreg_60_24.rename(columns = {"ever_dpd": "ever_dpd_60_24"})
aggreg_90_24  = aggreg_90_24.rename(columns = {"ever_dpd": "ever_dpd_90_24"})
aggreg_X_24  = aggreg_X_24.rename(columns = {"ever_dpd": "ever_dpd_X_24"})

In [57]:
df_mer_list = [aggreg_30_3,aggreg_60_3,aggreg_90_3,aggreg_X_3,
               aggreg_30_6,aggreg_60_6,aggreg_90_6,aggreg_X_6,
               aggreg_30_9,aggreg_60_9,aggreg_90_9,aggreg_X_9,
              aggreg_30_12,aggreg_60_12,aggreg_90_12,aggreg_X_12,
               aggreg_30_15,aggreg_60_15,aggreg_90_15,aggreg_X_15,
               aggreg_30_18,aggreg_60_18,aggreg_90_18,aggreg_X_18,
              aggreg_30_24,aggreg_60_24,aggreg_90_24,aggreg_X_24]

In [58]:
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['sz_loan_account_no'],
                                            how='outer'), df_mer_list)

In [59]:
df_merged.shape

(4460, 29)

In [60]:
df_merged[df_merged['ever_dpd_30_9'].notnull()].shape

(3988, 29)

In [129]:
df_merged.to_csv("D:/Varthana/all_perf.csv")